# Building the dataset of research papers

The [Entrez](http://biopython.org/DIST/docs/api/Bio.Entrez-module.html) module, a part of the [Biopython](http://biopython.org/) library, will be used to interface with [PubMed](http://www.ncbi.nlm.nih.gov/pubmed).<br>
You can download Biopython from [here](http://biopython.org/wiki/Download).

In this notebook we will be covering several of the steps taken in the [Biopython Tutorial](http://biopython.org/DIST/docs/tutorial/Tutorial.html), specifically in [Chapter 9  Accessing NCBI’s Entrez databases](http://biopython.org/DIST/docs/tutorial/Tutorial.html#htoc109).

In [1]:
from Bio import Entrez

# NCBI requires you to set your email address to make use of NCBI's E-utilities
Entrez.email = "Your.Name.Here@example.org"

The datasets will be saved as serialized Python objects, compressed with bzip2.
Saving/loading them will therefore require the [pickle](http://docs.python.org/3/library/pickle.html) and [bz2](http://docs.python.org/3/library/bz2.html) modules.

In [2]:
import pickle, bz2, os

## EInfo: Obtaining information about the Entrez databases

In [3]:
# accessing extended information about the PubMed database
pubmed = Entrez.read( Entrez.einfo(db="pubmed"), validate=False )[u'DbInfo']

# list of possible search fields for use with ESearch:
search_fields = { f['Name']:f['Description'] for f in pubmed["FieldList"] }

In search_fields, we find 'TIAB' ('Free text associated with Abstract/Title') as a possible search field to use in searches.

In [4]:
search_fields

{'AFFL': "Author's institutional affiliation and address",
 'ALL': 'All terms from all searchable fields',
 'AUCL': 'Author Cluster ID',
 'AUID': 'Author Identifier',
 'AUTH': 'Author(s) of publication',
 'BOOK': 'ID of the book that contains the document',
 'CDAT': 'Date of completion',
 'CNTY': 'Country of publication',
 'COIS': 'Conflict of Interest Statements',
 'COLN': 'Corporate Author of publication',
 'CRDT': 'Date publication first accessible through Entrez',
 'DSO': 'Additional text from the summary',
 'ECNO': 'EC number for enzyme or CAS registry number',
 'ED': "Section's Editor",
 'EDAT': 'Date publication first accessible through Entrez',
 'EID': 'Extended PMID',
 'EPDT': 'Date of Electronic publication',
 'FAUT': 'First Author of publication',
 'FILT': 'Limits the records',
 'FINV': 'Full name of investigator',
 'FULL': 'Full Author Name(s) of publication',
 'GRNT': 'NIH Grant Numbers',
 'INVR': 'Investigator',
 'ISBN': 'ISBN',
 'ISS': 'Issue number of publication',
 'JO

## ESearch: Searching the Entrez databases

To have a look at the kind of data we get when searching the database, we'll perform a search for papers authored by Haasdijk:

In [5]:
example_authors = ['Haasdijk E']
example_search = Entrez.read( Entrez.esearch( db="pubmed", term=' AND '.join([a+'[AUTH]' for a in example_authors]) ) )
example_search

{'TranslationStack': [{'Explode': 'N', 'Field': 'Author', 'Term': 'Haasdijk E[Author]', 'Count': '35'}, 'GROUP'], 'Count': '35', 'IdList': ['28513205', '28513201', '28323435', '28140628', '26933487', '24977986', '24901702', '24852945', '24708899', '24252306', '23580075', '23144668', '22174697', '22154920', '21870131', '21760539', '20662596', '20602234', '20386726', '18579581'], 'QueryTranslation': 'Haasdijk E[Author]', 'TranslationSet': [], 'RetStart': '0', 'RetMax': '20'}

Note how the result being produced is not in Python's native string format:

In [6]:
type( example_search['IdList'][0] )

Bio.Entrez.Parser.StringElement

The part of the query's result we are most interested in is accessible through

In [7]:
example_ids = [ int(id) for id in example_search['IdList'] ]
print(example_ids)

[28513205, 28513201, 28323435, 28140628, 26933487, 24977986, 24901702, 24852945, 24708899, 24252306, 23580075, 23144668, 22174697, 22154920, 21870131, 21760539, 20662596, 20602234, 20386726, 18579581]


### PubMed IDs dataset

We will now assemble a dataset comprised of research articles containing the keyword "evolution", in either their titles or abstracts.

In [8]:
search_term = 'aspirin'

In [9]:
Ids_file = 'data/' + search_term + '__Ids.pkl.bz2'

In [10]:
if os.path.exists( Ids_file ):
    Ids = pickle.load( bz2.BZ2File( Ids_file, 'rb' ) )
else:
    # determine the number of hits for the search term
    search = Entrez.read( Entrez.esearch( db="pubmed", term=search_term+'[TIAB]', retmax=0 ) )
    total = int( search['Count'] )
    
    # `Ids` will be incrementally assembled, by performing multiple queries,
    # each returning at most `retrieve_per_query` entries.
    Ids_str = []
    retrieve_per_query = 10000
    
    for start in range( 0, total, retrieve_per_query ):
        print('Fetching IDs of results [%d,%d]' % ( start, start+retrieve_per_query ) )
        s = Entrez.read( Entrez.esearch( db="pubmed", term=search_term+'[TIAB]', retstart=start, retmax=retrieve_per_query ) )
        Ids_str.extend( s[ u'IdList' ] )
    
    # convert Ids to integers (and ensure that the conversion is reversible)
    Ids = [ int(id) for id in Ids_str ]
    
    for (id_str, id_int) in zip(Ids_str, Ids):
        if str(id_int) != id_str:
            raise Exception('Conversion of PubMed ID %s from string to integer it not reversible.' % id_str )
    
    # Remove IDs that would cause problems below:
    Ids.remove(28334544)
    
    # Save list of Ids
    pickle.dump( Ids, bz2.BZ2File( Ids_file, 'wb' ) )
    
total = len( Ids )
print('%d documents contain the search term "%s".' % ( total, search_term ) )

43593 documents contain the search term "aspirin".


Taking a look at what we just retrieved, here are the last 5 elements of the `Ids` list:

In [11]:
Ids[:5]

[28981968, 28981961, 28978235, 28978007, 28976678]

## ESummary: Retrieving summaries from primary IDs

To have a look at the kind of metadata we get from a call to `Entrez.esummary()`, we now fetch the summary of one of Haasdijk's papers (using one of the PubMed IDs we obtained in the previous section:

In [12]:
example_paper = Entrez.read( Entrez.esummary(db="pubmed", id='28976678') )[0]

def print_dict( p ):
    for k,v in p.items():
        print(k)
        print('\t', v)

print_dict(example_paper)

Id
	 28976678
RecordStatus
	 PubMed
ISSN
	 0815-9319
EPubDate
	 
HasAbstract
	 1
References
	 []
PubDate
	 1998 Nov
ESSN
	 1440-1746
PubTypeList
	 ['Journal Article']
DOI
	 10.1111/j.1440-1746.1998.tb01876.x
Source
	 J Gastroenterol Hepatol
AuthorList
	 ['Konturek J', 'Konturek S', 'Dembinski A', 'Domschke W']
PubStatus
	 ppublish
SO
	 1998 Nov;13(S3):S193-S198
Issue
	 S3
PmcRefCount
	 0
ELocationID
	 doi: 10.1111/j.1440-1746.1998.tb01876.x
Pages
	 S193-S198
Title
	 Gastric adaptation to aspirin and Helicobacter pylori infection in man.
FullJournalName
	 Journal of gastroenterology and hepatology
History
	 {'medline': ['1998/11/01 00:01'], 'entrez': '2017/10/05 06:00', 'pubmed': ['1998/11/01 00:00']}
Volume
	 13
NlmUniqueID
	 8607909
LastAuthor
	 Domschke W
LangList
	 ['English']
Item
	 []
ArticleIds
	 {'medline': [], 'eid': '28976678', 'rid': '28976678', 'pubmed': ['28976678'], 'doi': '10.1111/j.1440-1746.1998.tb01876.x'}


For now, we'll keep just some basic information for each paper: title, list of authors, publication year, and [DOI](https://en.wikipedia.org/wiki/Digital_object_identifier).

In case you are not familiar with the DOI system, know that the paper above can be accessed through the link [http://dx.doi.org/10.1007/s12065-012-0071-x](http://dx.doi.org/10.1007/s12065-012-0071-x) (which is `http://dx.doi.org/` followed by the paper's DOI).

In [13]:
( example_paper['Title'], example_paper['AuthorList'], int(example_paper['PubDate'][:4]), example_paper['DOI'] )

('Gastric adaptation to aspirin and Helicobacter pylori infection in man.',
 ['Konturek J', 'Konturek S', 'Dembinski A', 'Domschke W'],
 1998,
 '10.1111/j.1440-1746.1998.tb01876.x')

### Summaries dataset

We are now ready to assemble a dataset containing the summaries of all the paper `Ids` we previously fetched.

To reduce the memory footprint, and to ensure the saved datasets won't depend on Biopython being installed to be properly loaded, values returned by `Entrez.read()` will be converted to their corresponding native Python types. We start by defining a function for helping with the conversion of strings:

In [14]:
Summaries_file = 'data/' + search_term + '__Summaries.pkl.bz2'

In [15]:
if os.path.exists( Summaries_file ):
    Summaries = pickle.load( bz2.BZ2File( Summaries_file, 'rb' ) )
else:
    # `Summaries` will be incrementally assembled, by performing multiple queries,
    # each returning at most `retrieve_per_query` entries.
    Summaries = []
    retrieve_per_query = 500
    
    print('Fetching Summaries of results: ')
    for start in range( 0, len(Ids), retrieve_per_query ):
        if (start % 10000 == 0):
            print('')
            print(start, end='')
        else:
            print('.', end='')
        
        # build comma separated string with the ids at indexes [start, start+retrieve_per_query)
        query_ids = ','.join( [ str(id) for id in Ids[ start : start+retrieve_per_query ] ] )
        
        s = Entrez.read( Entrez.esummary( db="pubmed", id=query_ids ) )
        
        # out of the retrieved data, we will keep only a tuple (title, authors, year, DOI), associated with the paper's id.
        # (all values converted to native Python formats)
        for p in s:
            try:
                f = [
                    ( int( p['Id'] ), (
                        str( p['Title'] ),
                        [ str(a) for a in p['AuthorList'] ],
                        int( p['PubDate'][:4] ),                # keeps just the publication year
                        str( p.get('DOI', '') )            # papers for which no DOI is available get an empty string in their place
                        ) )
                    ]
                Summaries.extend( f )
            except ValueError as e:
                print("\nError with ID " + p['Id'] + ": " + str(e))
                print("Manually remove this ID above and re-run code.")
    
    # Save Summaries, as a dictionary indexed by Ids
    Summaries = dict( Summaries )
    
    pickle.dump( Summaries, bz2.BZ2File( Summaries_file, 'wb' ) )

Let us take a look at the first 3 retrieved summaries:

In [16]:
{ id : Summaries[id] for id in Ids[:3] }

{28978235: ('The Effect of Monthly Medication on Mortality After a Coronary Event.',
  ['Montero-Balosa MC', 'Fernández-Urrusuno R', 'Vilches-Arenas A'],
  2017,
  '10.1177/1074248417732833'),
 28981961: ('Contemporary Use of Oral Antithrombotic Agents: Focus on Dual and Triple Therapeutic Approaches.',
  ['Jenkins AT', 'Kantorovich A', 'Burman L'],
  2017,
  '10.1002/phar.2041'),
 28981968: ('Omalizumab could be effective in children with severe eosinophilic non-allergic asthma.',
  ['Bourgoin-Heck M',
   'Amat F',
   'Trouvé C',
   'Bernard A',
   'Magny JP',
   'Lambert N',
   'Just J'],
  2017,
  '10.1111/pai.12813')}

## EFetch: Downloading full records from Entrez

`Entrez.efetch()` is the function that will allow us to obtain paper abstracts. Let us start by taking a look at the kind of data it returns when we query PubMed's database.

In [17]:
q = Entrez.read( Entrez.efetch(db="pubmed", id='28976678', retmode="xml") )['PubmedArticle']

`q` is a list, with each member corresponding to a queried id. Because here we only queried for one id, its results are then in `q[0]`.

In [18]:
type(q), len(q)

(list, 1)

1
At `q[0]` we find a dictionary containing two keys, the contents of which we print below.

In [19]:
type(q[0]), q[0].keys()

(Bio.Entrez.Parser.DictionaryElement,
 dict_keys(['MedlineCitation', 'PubmedData']))

In [20]:
print_dict( q[0][ 'PubmedData' ] )

ArticleIdList
	 [StringElement('28976678', attributes={'IdType': 'pubmed'}), StringElement('10.1111/j.1440-1746.1998.tb01876.x', attributes={'IdType': 'doi'})]
PublicationStatus
	 ppublish
History
	 [DictElement({'Year': '2017', 'Hour': '6', 'Minute': '0', 'Month': '10', 'Day': '5'}, attributes={'PubStatus': 'entrez'}), DictElement({'Year': '1998', 'Hour': '0', 'Minute': '0', 'Month': '11', 'Day': '1'}, attributes={'PubStatus': 'pubmed'}), DictElement({'Year': '1998', 'Hour': '0', 'Minute': '1', 'Month': '11', 'Day': '1'}, attributes={'PubStatus': 'medline'})]


The key `'MedlineCitation'` maps into another dictionary. In that dictionary, most of the information is contained under the key `'Article'`. To minimize the clutter, below we show the contents of `'MedlineCitation'` excluding its `'Article'` member, and below that we then show the contents of `'Article'`.

In [21]:
print_dict( { k:v for k,v in q[0][ 'MedlineCitation' ].items() if k!='Article' } )

CitationSubset
	 []
KeywordList
	 [ListElement([StringElement('Helicobacter pylori', attributes={'MajorTopicYN': 'N'}), StringElement('adaptation', attributes={'MajorTopicYN': 'N'}), StringElement('aspirin', attributes={'MajorTopicYN': 'N'}), StringElement('epithelial cell proliferation', attributes={'MajorTopicYN': 'N'}), StringElement('gastric blood flow', attributes={'MajorTopicYN': 'N'}), StringElement('stomach', attributes={'MajorTopicYN': 'N'})], attributes={'Owner': 'NOTNLM'})]
OtherID
	 []
PMID
	 28976678
DateRevised
	 {'Year': '2017', 'Month': '10', 'Day': '04'}
MedlineJournalInfo
	 {'Country': 'Australia', 'NlmUniqueID': '8607909', 'MedlineTA': 'J Gastroenterol Hepatol', 'ISSNLinking': '0815-9319'}
SpaceFlightMission
	 []
DateCreated
	 {'Year': '2017', 'Month': '10', 'Day': '04'}
OtherAbstract
	 []
GeneralNote
	 []


In [22]:
print_dict( q[0][ 'MedlineCitation' ][ 'Article' ] )

Journal
	 {'Title': 'Journal of gastroenterology and hepatology', 'JournalIssue': DictElement({'Issue': 'S3', 'PubDate': {'Year': '1998', 'Month': 'Nov'}, 'Volume': '13'}, attributes={'CitedMedium': 'Internet'}), 'ISSN': StringElement('1440-1746', attributes={'IssnType': 'Electronic'}), 'ISOAbbreviation': 'J. Gastroenterol. Hepatol.'}
ELocationID
	 [StringElement('10.1111/j.1440-1746.1998.tb01876.x', attributes={'ValidYN': 'Y', 'EIdType': 'doi'})]
Pagination
	 {'MedlinePgn': 'S193-S198'}
ArticleDate
	 []
AuthorList
	 ListElement([DictElement({'Identifier': [], 'Initials': 'J', 'AffiliationInfo': [{'Identifier': [], 'Affiliation': 'Department of Medicine B, University of Münster, Münster, GermanyInstitute of Physiology, and Pathomorphology, Jagiellonian University, Krakow, Poland.'}], 'ForeName': 'Jan', 'LastName': 'Konturek'}, attributes={'ValidYN': 'Y'}), DictElement({'Identifier': [], 'Initials': 'S', 'AffiliationInfo': [{'Identifier': [], 'Affiliation': 'Department of Medicine B, Un

A paper's abstract can therefore be accessed with:

In [23]:
{ int(q[0]['MedlineCitation']['PMID']) : str(q[0]['MedlineCitation']['Article']['Abstract']['AbstractText'][0]) }

{28976678: 'The relationship between Helicobacter pylori infection and aspirin (ASA)-induced gastropathy and gastric adaptation to ASA remains unclear. We compared gastric damage and adaptation after repeated exposures to ASA in the same subjects without H. pylori infection and those infected by H. pylori before and after eradication of this H. pylori. Twenty-four volunteers in two groups (A and B), without H. pylori infection (group A) and with H. pylori infection (group B) before and after H. pylori eradication, were given ASA 2 g/day or placebo for 14 days. Mucosal damage was evaluated by endoscopy and gastric microbleeding; mucosal prostaglandin (PG) E2 generation and luminal transforming growth factor (TGF)α were determined on days 0,3,7 and 14 of the ASA course. In all subjects, ASA-induced gastric damage reached a maximum on day 3. In H. pylori-positive subjects this damage was maintained at a similar level up to the 14th day of observation. Following H. pylori eradication, the 

Some of the ids in our dataset refer to books from the [NCBI Bookshelf](http://www.ncbi.nlm.nih.gov/books/), a collection of freely available, downloadable, on-line versions of selected biomedical books. For such ids, `Entrez.efetch()` returns a slightly different structure, where the keys `[u'BookDocument', u'PubmedBookData']` take the place of the `[u'MedlineCitation', u'PubmedData']` keys we saw above.

### Abstracts dataset

We can now assemble a dataset mapping paper ids to their abstracts.

In [24]:
Abstracts_file = 'data/' + search_term + '__Abstracts.pkl.bz2'

In [25]:
import http.client
from collections import deque

if os.path.exists( Abstracts_file ):
    Abstracts = pickle.load( bz2.BZ2File( Abstracts_file, 'rb' ) )
else:
    # `Abstracts` will be incrementally assembled, by performing multiple queries,
    # each returning at most `retrieve_per_query` entries.
    Abstracts = deque()
    retrieve_per_query = 500
    
    print('Fetching Abstracts of results: ')
    for start in range( 0, len(Ids), retrieve_per_query ):
        if (start % 10000 == 0):
            print('')
            print(start, end='')
        else:
            print('.', end='')
        
        # build comma separated string with the ids at indexes [start, start+retrieve_per_query)
        query_ids = ','.join( [ str(id) for id in Ids[ start : start+retrieve_per_query ] ] )
        
        # issue requests to the server, until we get the full amount of data we expect
        while True:
            try:
                s = Entrez.read( Entrez.efetch(db="pubmed", id=query_ids, retmode="xml" ) )['PubmedArticle']
            except http.client.IncompleteRead:
                print('r', end='')
                continue
            break
        
        i = 0
        for p in s:
            abstr = ''
            if 'MedlineCitation' in p:
                pmid = p['MedlineCitation']['PMID']
                if 'Abstract' in p['MedlineCitation']['Article']:
                    abstr = p['MedlineCitation']['Article']['Abstract']['AbstractText'][0]
            elif 'BookDocument' in p:
                pmid = p['BookDocument']['PMID']
                if 'Abstract' in p['BookDocument']:
                    abstr = p['BookDocument']['Abstract']['AbstractText'][0]
            else:
                raise Exception('Unrecognized record type, for id %d (keys: %s)' % (Ids[start+i], str(p.keys())) )
            
            Abstracts.append( (int(pmid), str(abstr)) )
            i += 1
    
    # Save Abstracts, as a dictionary indexed by Ids
    Abstracts = dict( Abstracts )
    
    pickle.dump( Abstracts, bz2.BZ2File( Abstracts_file, 'wb' ) )

Taking a look at one paper's abstract:

In [26]:
Abstracts[28976678]

'The relationship between Helicobacter pylori infection and aspirin (ASA)-induced gastropathy and gastric adaptation to ASA remains unclear. We compared gastric damage and adaptation after repeated exposures to ASA in the same subjects without H. pylori infection and those infected by H. pylori before and after eradication of this H. pylori. Twenty-four volunteers in two groups (A and B), without H. pylori infection (group A) and with H. pylori infection (group B) before and after H. pylori eradication, were given ASA 2 g/day or placebo for 14 days. Mucosal damage was evaluated by endoscopy and gastric microbleeding; mucosal prostaglandin (PG) E2 generation and luminal transforming growth factor (TGF)α were determined on days 0,3,7 and 14 of the ASA course. In all subjects, ASA-induced gastric damage reached a maximum on day 3. In H. pylori-positive subjects this damage was maintained at a similar level up to the 14th day of observation. Following H. pylori eradication, the damage was 

## ELink: Searching for related items in NCBI Entrez

To understand how to obtain paper citations with Entrez, we will first assemble a small set of PubMed IDs, and then query for their citations.
To that end, we search here for papers published in the PLoS ONE journal (as before, having also the word "aspirin" in either the title or abstract):

In [27]:
CA_search_term = search_term+'[TIAB] AND PLoS ONE[JOUR]'
CA_ids = Entrez.read( Entrez.esearch( db="pubmed", term=CA_search_term ) )['IdList']
CA_ids

['28968454', '28931015', '28910305', '28880908', '28863196', '28841657', '28813512', '28792522', '28715503', '28636624', '28542447', '28542316', '28493889', '28475584', '28472145', '28459825', '28453510', '28426793', '28403216', '28362840']

In [28]:
CA_summ = {
    p['Id'] : ( p['Title'], p['AuthorList'], p['PubDate'][:4], p['FullJournalName'], p.get('DOI', '') )
    for p in Entrez.read( Entrez.esummary(db="pubmed", id=','.join( CA_ids )) )
    }
CA_summ

{'28362840': ('Aspirin-triggered resolvin D1 attenuates PDGF-induced vascular smooth muscle cell migration via the cyclic adenosine monophosphate/protein kinase A (cAMP/PKA) pathway.',
  ['Mottola G', 'Chatterjee A', 'Wu B', 'Chen M', 'Conte MS'],
  '2017',
  'PloS one',
  '10.1371/journal.pone.0174936'),
 '28403216': ('Efficacy and safety of aspirin in patients with peripheral vascular disease: An updated systematic review and meta-analysis of randomized controlled trials.',
  ['Mahmoud AN', 'Elgendy AY', 'Rambarat C', 'Mahtta D', 'Elgendy IY', 'Bavry AA'],
  '2017',
  'PloS one',
  '10.1371/journal.pone.0175283'),
 '28426793': ('Prevalence of cardiovascular medication on secondary prevention after myocardial infarction in China between 1995-2015: A systematic review and meta-analysis.',
  ['Zhao M', 'Klipstein-Grobusch K', 'Wang X', 'Reitsma JB', 'Zhao D', 'Grobbee DE', 'Graham I', 'Vaartjes I'],
  '2017',
  'PloS one',
  '10.1371/journal.pone.0175947'),
 '28453510': ('Is aspirin ass

Because we restricted our search to papers in an open-access journal, you can then follow their DOIs to freely access their PDFs at the journal's website.

We will now issue calls to `Entrez.elink()` using these PubMed IDs, to retrieve the IDs of papers that cite them.
The database from which the IDs will be retrieved is [PubMed Central](http://www.ncbi.nlm.nih.gov/pmc/), a free digital database of full-text scientific literature in the biomedical and life sciences.

A complete list of the kinds of links you can retrieve with `Entrez.elink()` can be found [here](http://eutils.ncbi.nlm.nih.gov/entrez/query/static/entrezlinks.html).

In [29]:
CA_citing = {
    id : Entrez.read( Entrez.elink(
            cmd = "neighbor",               # ELink command mode: "neighbor", returns
                                            #     a set of UIDs in `db` linked to the input UIDs in `dbfrom`.
            dbfrom = "pubmed",              # Database containing the input UIDs: PubMed
            db = "pmc",                     # Database from which to retrieve UIDs: PubMed Central
            LinkName = "pubmed_pmc_refs",   # Name of the Entrez link to retrieve: "pubmed_pmc_refs", gets
                                            #     "Full-text articles in the PubMed Central Database that cite the current articles"
            from_uid = id                   # input UIDs
            ) )
    for id in CA_ids
    }

CA_citing['28792522']

[{'IdList': ['28792522'], 'ERROR': [], 'LinkSetDbHistory': [], 'DbFrom': 'pubmed', 'LinkSetDb': [{'Link': [{'Id': '5602518'}], 'DbTo': 'pmc', 'LinkName': 'pubmed_pmc_refs'}]}]

We have in `CA_citing[paper_id][0]['LinkSetDb'][0]['Link']` the list of papers citing `paper_id`. To get it as just a list of ids, we can do

In [30]:
cits = [ l['Id'] for l in CA_citing['28792522'][0]['LinkSetDb'][0]['Link'] ]
cits

['5602518']

However, one more step is needed, as what we have now are PubMed Central IDs, and not PubMed IDs. Their conversion can be achieved through an additional call to `Entrez.elink()`:

In [31]:
cits_pm = Entrez.read( Entrez.elink( dbfrom="pmc", db="pubmed", LinkName="pmc_pubmed", from_uid=",".join(cits)) )
cits_pm

[{'IdList': ['5602518'], 'ERROR': [], 'LinkSetDbHistory': [], 'DbFrom': 'pmc', 'LinkSetDb': [{'Link': [{'Id': '28922365'}], 'DbTo': 'pubmed', 'LinkName': 'pmc_pubmed'}]}]

In [32]:
ids_map = { pmc_id : link['Id'] for (pmc_id,link) in zip(cits_pm[0]['IdList'], cits_pm[0]['LinkSetDb'][0]['Link']) }
ids_map

{'5602518': '28922365'}

And to check these papers:

In [33]:
{   p['Id'] : ( p['Title'], p['AuthorList'], p['PubDate'][:4], p['FullJournalName'], p.get('DOI', '') )
    for p in Entrez.read( Entrez.esummary(db="pubmed", id=','.join( ids_map.values() )) )
    }

{'28922365': ('Isothiocyanate-enriched moringa seed extract alleviates ulcerative colitis symptoms in mice.',
  ['Kim Y', 'Wu AG', 'Jaja-Chimedza A', 'Graf BL', 'Waterman C', 'Verzi MP', 'Raskin I'],
  '2017',
  'PloS one',
  '10.1371/journal.pone.0184709')}

### Citations dataset

We have now seen all the steps required to assemble a dataset of citations to each of the papers in our dataset.

In [34]:
Citations_file = 'data/' + search_term + '__Citations.pkl.bz2'
Citations = []

At least one server query will be issued per paper in `Ids`. Because NCBI allows for at most 3 queries per second (see [here](http://biopython.org/DIST/docs/api/Bio.Entrez-pysrc.html#_open)), this dataset will take a long time to assemble. Should you need to interrupt it for some reason, or the connection fail at some point, it is safe to just rerun the cell below until all data is collected.

In [42]:
import http.client

if Citations == [] and os.path.exists( Citations_file ):
    Citations = pickle.load( bz2.BZ2File( Citations_file, 'rb' ) )

if len(Citations) < len(Ids):
    
    i = len(Citations)
    checkpoint = len(Ids) / 10 + 1      # save to hard drive at every 10% of Ids fetched
    
    for pm_id in Ids[i:]:               # either starts from index 0, or resumes from where we previously left off
        
        while True:
            try:
                # query for papers archived in PubMed Central that cite the paper with PubMed ID `pm_id`
                c = Entrez.read( Entrez.elink( dbfrom = "pubmed", db="pmc", LinkName = "pubmed_pmc_refs", id=str(pm_id) ) )
                
                c = c[0]['LinkSetDb']
                if len(c) == 0:
                    # no citations found for the current paper
                    c = []
                else:
                    c = [ l['Id'] for l in c[0]['Link'] ]
                    
                    # convert citations from PubMed Central IDs to PubMed IDs
                    p = []
                    retrieve_per_query = 500
                    for start in range( 0, len(c), retrieve_per_query ):
                        query_ids = ','.join( c[start : start+retrieve_per_query] )
                        r = Entrez.read( Entrez.elink( dbfrom="pmc", db="pubmed", LinkName="pmc_pubmed", from_uid=query_ids ) )
                        # select the IDs. If no matching PubMed ID was found, [] is returned instead
                        p.extend( [] if r[0]['LinkSetDb']==[] else [ int(link['Id']) for link in r[0]['LinkSetDb'][0]['Link'] ] )
                    c = p
            
            except http.client.BadStatusLine:
                # Presumably, the server closed the connection before sending a valid response. Retry until we have the data.
                print('r')
                continue
            break
        
        Citations.append( (pm_id, c) )
        if (i % 10000 == 0):
            print('')
            print(i, end='')
        if (i % 100 == 0):
            print('.', end='')
        i += 1
        
        if i % checkpoint == 0:
            print('\tsaving at checkpoint', i)
            pickle.dump( Citations, bz2.BZ2File( Citations_file, 'wb' ) )
    
    print('\n done.')
    
    # Save Citations, as a dictionary indexed by Ids
    Citations = dict( Citations )
    
    pickle.dump( Citations, bz2.BZ2File( Citations_file, 'wb' ) )

...................................................
20000....................................................................................................
30000....................................................................................................
40000....................................
 done.


To see that we have indeed obtained the data we expected, you can match the ids below, with the ids listed at the end of last section.

In [47]:
Citations[28792522]

[28922365]

## Where do we go from here?

Running the code above generates multiple local files, containing the datasets we'll be working with. Loading them into memory is a matter of just issuing a call like<br>
``data = pickle.load( bz2.BZ2File( data_file, 'rb' ) )``.

The Entrez module will therefore no longer be needed, unless you wish to extend your data processing with additional information retrieved from PubMed.

Should you be interested in looking at alternative ways to handle the data, have a look at the [sqlite3](http://docs.python.org/3/library/sqlite3.html) module included in Python's standard library, or [Pandas](http://pandas.pydata.org/), the Python Data Analysis Library.